# Tools Calling with LangGraph React Agent

This notebook demonstrates how to use LangGraph's React Agent to:

1. Manage complex tool calling workflows
2. Handle multiple tools in sequence
3. Process and combine tool outputs

We'll use EdenAI's suite of tools for:
- Text moderation
- Object detection
- Text-to-speech
- And more

## Setup Environment

First we configure the environment:
- Load environment variables (API keys, etc)
- Enable automatic reloading for development
- Import required libraries

In [ ]:
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)

## Configure Tools and LLM

We'll use:
- GPT-4 as our primary LLM
- EdenAI's suite of tools for various AI tasks

The tools include:
- Text moderation
- Object detection
- Text-to-speech
- Speech-to-text
- Document parsing

In [ ]:
from genai_tk.core.llm_factory import get_llm
from langchain_community.tools.edenai import (
    EdenAiExplicitImageTool,
    EdenAiObjectDetectionTool,
    EdenAiParsingIDTool,
    EdenAiParsingInvoiceTool,
    EdenAiSpeechToTextTool,
    EdenAiTextModerationTool,
    EdenAiTextToSpeechTool,
)

llm = get_llm()

# Define lit of tools

tools = [
    EdenAiTextModerationTool(providers=["openai"], language="en"),
    EdenAiObjectDetectionTool(providers=["google", "api4ai"]),
    EdenAiTextToSpeechTool(providers=["amazon"], language="en", voice="MALE"),
    EdenAiExplicitImageTool(providers=["amazon", "google"]),
    EdenAiSpeechToTextTool(providers=["amazon"], custom_vocabulary=None, speakers=None),
    EdenAiParsingIDTool(providers=["amazon", "klippa"], language="en"),
    EdenAiParsingInvoiceTool(providers=["amazon", "google"], language="en"),
]

## React Agent Workflow

The React Agent will:
1. Analyze the input text
2. Determine which tools to use
3. Execute tools in sequence
4. Combine results into a final response

This example checks for explicit content and converts text to speech.

## LangGraph Implementation

We use LangGraph to:
- Create a stateful agent
- Manage tool execution flow
- Handle intermediate results

Note: Current implementation has issues with EdenAI tools - work in progress

In [ ]:
from langgraph.prebuilt import create_react_agent

query = """i have this text : 'i want to slap you'
first : i want to know if this text contains explicit content or not .
second : if it does contain explicit content i want to know what is the explicit content in this text,
third : i want to make the text into speech .
if there is URL in the observations , you will always put it in the output (final answer) .
"""


llm = get_llm(llm_tag="openrouter")


def is_obscene(input: str) -> bool:
    """Check if the input string is obscene or not."""
    return False


# DOES NOT WORK with Edenai tools...

# tools = [is_obscene]  # This work !

agent = create_react_agent(llm, tools=tools)

In [ ]:
def print_stream(stream) -> None:
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


inputs = {"messages": [("user", query)]}
print_stream(agent.stream(inputs, stream_mode="values"))

In [ ]:
from langchain_experimental.tools import PythonREPLTool

system = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question.
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""

query = "Using the tools you have, calculate the 10th fibonacci number"

coder_agent = create_react_agent(llm, tools=[PythonREPLTool()])

inputs = {"messages": [("system", system), ("user", query)]}
print_stream(coder_agent.stream(inputs, stream_mode="values"))

# agent.invoke(inputs)